## Dependencies

In [1]:
from tqdm import tqdm
import statistics

import torch
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score

import torch_geometric.transforms as T
from torch_geometric.datasets import SNAPDataset
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, GINConv
from torch_geometric.utils import negative_sampling, to_networkx

torch.manual_seed(0)

%matplotlib notebook

C:\Users\Abhijit\Documents\GitHub\cpsc490\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

orig_transform = T.Compose([
    T.ToDevice(device),
    T.RemoveIsolatedNodes(),
])

transform = T.Compose([
    T.ToDevice(device),
    T.RemoveIsolatedNodes(),
    T.RandomLinkSplit(
        num_val=0.05, 
        num_test=0.1, 
        is_undirected=True, 
        add_negative_train_samples=False
    ),
])

orig_dataset = SNAPDataset(root="../data/SNAPDataset", name="ego-facebook", transform=orig_transform)
dataset = SNAPDataset(root="../data/SNAPDataset", name="ego-facebook", transform=transform)

data = orig_dataset[0]
train_data, val_data, test_data = dataset[0]

In [3]:
print(data)
print(train_data)
print(val_data)
print(test_data)

Data(x=[347, 1406], edge_index=[2, 5718], circle=[325], circle_batch=[325])
Data(x=[347, 1406], edge_index=[2, 4864], circle=[325], circle_batch=[325], edge_label=[2432], edge_label_index=[2, 2432])
Data(x=[347, 1406], edge_index=[2, 4864], circle=[325], circle_batch=[325], edge_label=[284], edge_label_index=[2, 284])
Data(x=[347, 1406], edge_index=[2, 5148], circle=[325], circle_batch=[325], edge_label=[570], edge_label_index=[2, 570])


## Prediction

In [4]:
from torch import nn
import torch.nn.functional as F


class SimpleNet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)
    
    def forward(self, x, edge_index, data=None):
        z = self.encode(x, edge_index)
        out = model.decode(z, edge_index)
        return torch.hstack((-out, out)).T


class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        # TODO: look into SAGEConv, GATConv, GINConv, comparison between
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        
        self.W1 = nn.Linear(out_channels * 2, out_channels)
        self.W2 = nn.Linear(out_channels, 1)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        z1 = torch.cat((z[edge_label_index[0]], z[edge_label_index[1]]), dim=1)
        out1 = self.W2(F.relu(self.W1(z1)).squeeze())
        
        z2 = torch.cat((z[edge_label_index[1]], z[edge_label_index[0]]), dim=1)
        out2 = self.W2(F.relu(self.W1(z2)).squeeze())
        
        return (out1 + out2) / 2
    
    def forward(self, x, edge_index, edge_label_index, data=None):
        z = self.encode(x, edge_index)
        out = model.decode(z, edge_label_index)
        return torch.hstack((-out, out)).T

simple_model = SimpleNet(dataset.num_features, 128, 32).to(device)
simple_optimizer = torch.optim.Adam(params=simple_model.parameters(), lr=3e-3)
    
model = Net(dataset.num_features, 128, 32).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=3e-3)
criterion = torch.nn.BCEWithLogitsLoss()

# TODO: These methods simultaneously use node feature and graph structure properties.
#       Is it possible to train models that look at each aspect separately
#       Can look at only node features by just passing original layer to MLP
#       Unsure if can look at only graph by passing random vector into GCNConv
#       Should also read up on Node2Vec and other methods of generating node embeddings (talk to Rex)

In [5]:
def train(model, optimizer, data):
    model.train()
    optimizer.zero_grad()
    z = model.encode(data.x, data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=data.edge_index, 
        num_nodes=data.num_nodes,
        num_neg_samples=data.edge_label_index.shape[1], 
        method='sparse'
    )
    
    edge_label_index = torch.cat([data.edge_label_index, neg_edge_index], dim=-1)
    edge_label = torch.cat([data.edge_label, data.edge_label.new_zeros(neg_edge_index.size(1))], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(model, data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    a, b = data.edge_label.cpu().numpy(), out.cpu().numpy()
    c = (out > 0.5).float().cpu().numpy()
    return roc_auc_score(a, b), accuracy_score(a, c)

In [6]:
best_val_auc = final_test_auc = final_test_acc = 0
for epoch in range(1, 1001):
    loss = train(simple_model, simple_optimizer, train_data)
    val_auc, val_acc = test(simple_model, val_data)
    test_auc, test_acc = test(simple_model, test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
        final_test_acc = test_acc
    if epoch % 50 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f} {val_acc:.4f}, Test: {test_auc:.4f} {test_acc:.4f}')

print(f'Final Test: {final_test_auc:.4f} {final_test_acc:.4f}')

simple_z = simple_model.encode(test_data.x, test_data.edge_index)
simple_final_edge_index = simple_model.decode(simple_z, test_data.edge_label_index)

Epoch: 050, Loss: 0.4866, Val: 0.9183 0.7606, Test: 0.8872 0.7579
Epoch: 100, Loss: 0.4678, Val: 0.9312 0.7676, Test: 0.8991 0.7772
Epoch: 150, Loss: 0.4449, Val: 0.9192 0.7711, Test: 0.8975 0.7842
Epoch: 200, Loss: 0.4395, Val: 0.9217 0.7465, Test: 0.9005 0.7825
Epoch: 250, Loss: 0.4440, Val: 0.9219 0.7641, Test: 0.8979 0.8000
Epoch: 300, Loss: 0.4322, Val: 0.9169 0.7887, Test: 0.8980 0.8070
Epoch: 350, Loss: 0.4283, Val: 0.9085 0.7852, Test: 0.8861 0.8035
Epoch: 400, Loss: 0.4303, Val: 0.9168 0.7923, Test: 0.8887 0.7947
Epoch: 450, Loss: 0.4392, Val: 0.9081 0.8063, Test: 0.8839 0.7895
Epoch: 500, Loss: 0.4150, Val: 0.9058 0.7993, Test: 0.8815 0.7912
Epoch: 550, Loss: 0.4118, Val: 0.9104 0.8099, Test: 0.8800 0.7895
Epoch: 600, Loss: 0.4303, Val: 0.9022 0.7993, Test: 0.8788 0.7947
Epoch: 650, Loss: 0.4222, Val: 0.9124 0.7993, Test: 0.8870 0.7895
Epoch: 700, Loss: 0.4206, Val: 0.8965 0.7958, Test: 0.8744 0.7912
Epoch: 750, Loss: 0.4045, Val: 0.8963 0.8063, Test: 0.8726 0.7947
Epoch: 800

In [7]:
best_val_auc = final_test_auc = final_test_acc = 0
for epoch in range(1, 1001):
    loss = train(model, optimizer, train_data)
    val_auc, val_acc = test(model, val_data)
    test_auc, test_acc = test(model, test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
        final_test_acc = test_acc
    if epoch % 50 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f} {val_acc:.4f}, Test: {test_auc:.4f} {test_acc:.4f}')

print(f'Final Test: {final_test_auc:.4f} {final_test_acc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode(z, test_data.edge_label_index)

Epoch: 050, Loss: 0.3929, Val: 0.9296 0.8627, Test: 0.8937 0.8105
Epoch: 100, Loss: 0.3120, Val: 0.9285 0.8486, Test: 0.9290 0.8439
Epoch: 150, Loss: 0.2919, Val: 0.9344 0.8627, Test: 0.9385 0.8649
Epoch: 200, Loss: 0.2823, Val: 0.9412 0.8662, Test: 0.9393 0.8789
Epoch: 250, Loss: 0.2697, Val: 0.9449 0.8732, Test: 0.9408 0.8737
Epoch: 300, Loss: 0.2831, Val: 0.9453 0.8697, Test: 0.9407 0.8719
Epoch: 350, Loss: 0.2687, Val: 0.9461 0.8768, Test: 0.9420 0.8649
Epoch: 400, Loss: 0.2513, Val: 0.9454 0.8979, Test: 0.9419 0.8807
Epoch: 450, Loss: 0.2605, Val: 0.9485 0.8803, Test: 0.9426 0.8772
Epoch: 500, Loss: 0.2467, Val: 0.9463 0.8838, Test: 0.9440 0.8825
Epoch: 550, Loss: 0.2347, Val: 0.9457 0.8873, Test: 0.9439 0.8877
Epoch: 600, Loss: 0.2349, Val: 0.9477 0.8732, Test: 0.9439 0.8754
Epoch: 650, Loss: 0.2360, Val: 0.9488 0.8803, Test: 0.9445 0.8825
Epoch: 700, Loss: 0.2519, Val: 0.9476 0.8732, Test: 0.9438 0.8737
Epoch: 750, Loss: 0.2307, Val: 0.9473 0.8838, Test: 0.9440 0.8825
Epoch: 800

In [8]:
(test_data.edge_label == (torch.sigmoid(simple_final_edge_index.squeeze()) > 0.7)).sum() / test_data.edge_label.shape[0]

tensor(0.8544)

In [9]:
(test_data.edge_label == (torch.sigmoid(final_edge_index.squeeze()) > 0.5)).sum() / test_data.edge_label.shape[0]

tensor(0.8842)

In [10]:
simple_model.forward(test_data.x, test_data.edge_index, test_data.edge_label_index)

tensor([[-0.7007,  0.1209, -0.5373,  ...,  1.3765, -0.4524,  0.1600],
        [ 0.7007, -0.1209,  0.5373,  ..., -1.3765,  0.4524, -0.1600]],
       grad_fn=<PermuteBackward0>)

In [11]:
model.forward(test_data.x, test_data.edge_index, test_data.edge_label_index)

tensor([[ -8.1210,  -0.9013,  -0.9967,  ...,  -1.3552,  13.1518,   6.6645],
        [  8.1210,   0.9013,   0.9967,  ...,   1.3552, -13.1518,  -6.6645]],
       grad_fn=<PermuteBackward0>)

## SubgraphX

In [78]:
from datetime import datetime

In [79]:
test_data.edge_label_index[:, 12]

tensor([ 24, 187])

In [81]:
node_1 = 24
node_2 = 187

node_1_neighbors = set(test_data.edge_index[:, (test_data.edge_index[0] == node_1)][1].cpu().numpy())
node_2_neighbors = set(test_data.edge_index[:, (test_data.edge_index[0] == node_2)][1].cpu().numpy())
neighbors = np.array(list(node_1_neighbors.union(node_2_neighbors)))

print("Num Neighbors:", neighbors.shape[0])
print("Node 1 Neighbors:", node_1_neighbors)
print("Node 2 Neighbors:", node_2_neighbors)
print("Overlapping Neighbors:", node_1_neighbors.intersection(node_2_neighbors))

Num Neighbors: 81
Node 1 Neighbors: {256, 129, 2, 269, 270, 271, 275, 147, 276, 20, 279, 25, 282, 157, 30, 289, 290, 38, 39, 168, 169, 296, 175, 50, 55, 184, 185, 64, 321, 66, 322, 324, 68, 198, 71, 199, 72, 330, 75, 331, 335, 82, 83, 87, 346, 220, 93, 223, 97, 230, 231, 103, 102, 104, 235, 236, 238, 112, 245, 121}
Node 2 Neighbors: {257, 2, 132, 8, 12, 141, 270, 271, 273, 276, 20, 279, 25, 155, 284, 29, 289, 290, 164, 296, 169, 303, 175, 55, 184, 185, 314, 321, 322, 66, 199, 74, 333, 78, 207, 84, 341, 222, 103, 238, 112, 118, 121, 251}
Overlapping Neighbors: {2, 270, 271, 276, 20, 279, 25, 289, 290, 296, 169, 175, 55, 184, 185, 321, 322, 66, 199, 103, 238, 112, 121}


In [84]:
start = datetime.now()

T = 100
for neighbor in neighbors:
    pred_diffs = []
    for t in range(T):
        S_filter = torch.zeros(test_data.edge_index.shape[1], dtype=bool)
        S_filter[sub_edge_mask] = True
        S_filter[(sub_edge_mask) & (np.random.random(sub_edge_mask.shape[0]) > 0.5)] = False
        S_filter[(test_data.edge_index[0] == neighbor)] = False
        
        old_z = model.encode(test_data.x, test_data.edge_index[:, S_filter])
        old_pred = model.decode(old_z, torch.tensor([[node_1], [node_2]]))
        
        S_filter[(test_data.edge_index[0] == neighbor)] = True
        new_z = model.encode(test_data.x, test_data.edge_index[:, S_filter])
        new_pred = model.decode(new_z, torch.tensor([[node_1], [node_2]]))
        
        pred_diff = (new_pred - old_pred)
        pred_diffs.append(pred_diff.item())
    diff_avg, diff_std = sum(pred_diffs) / len(pred_diffs), statistics.stdev(pred_diffs) / np.sqrt(T)
    print(neighbor, "\t", round(diff_avg, 5), "\t", round(diff_std, 5), "\t", round(diff_avg / diff_std, 5))
    
print(datetime.now() - start)

C:\Users\Abhijit\AppData\Local\Temp\ipykernel_4984\888700931.py:9: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorAdvancedIndexing.cpp:1582.)
  S_filter[(sub_edge_mask) & (np.random.random(sub_edge_mask.shape[0]) > 0.5)] = False


256 	 -0.23624 	 0.20532 	 -1.15057
257 	 2.63194 	 0.1327 	 19.8338
2 	 1.70449 	 0.1264 	 13.48506
8 	 -0.25916 	 0.30501 	 -0.84968
12 	 2.26551 	 0.23972 	 9.45075
269 	 -2.0956 	 0.09302 	 -22.52953
270 	 11.00256 	 0.34759 	 31.65416
271 	 1.46325 	 0.37257 	 3.92751
273 	 0.73915 	 0.14742 	 5.01373
275 	 6.56457 	 0.22725 	 28.8874
276 	 7.04733 	 0.18199 	 38.72284
20 	 16.52218 	 0.32587 	 50.70237
279 	 0.19623 	 0.21007 	 0.93415
25 	 -2.41853 	 0.25051 	 -9.65425
282 	 -0.08848 	 0.09243 	 -0.95724
284 	 11.34167 	 0.35969 	 31.53145
29 	 -2.04181 	 0.11809 	 -17.28971
30 	 2.33152 	 0.17115 	 13.62265
289 	 -0.21164 	 0.12535 	 -1.68838
290 	 1.23024 	 0.2617 	 4.70092
38 	 -0.98675 	 0.10734 	 -9.19234
39 	 7.18512 	 0.24824 	 28.94441
296 	 -2.37144 	 0.22017 	 -10.77106
303 	 -9.93497 	 0.23562 	 -42.16504
50 	 -5.49272 	 0.15114 	 -36.34202
55 	 -4.00812 	 0.30177 	 -13.28219
314 	 -1.94736 	 0.2471 	 -7.88075
64 	 1.75813 	 0.10181 	 17.26805
321 	 -4.61022 	 0.21631

In [85]:
from torch_geometric.data import Data

start = datetime.now()

T = 100
data_arr = []
for neighbor in neighbors: 
    temp_edge_label_index = torch.tensor([[node_1], [node_2]])
    # Only optimizes the edges from neighbors to node_1/node_2, other direction not needed for prediction
    sub_edge_mask = (test_data.edge_index[1] == node_1) |  (test_data.edge_index[1] == node_2)
    pred_diffs = []
    for t in range(T):
        S_filter = torch.zeros(test_data.edge_index.shape[1], dtype=bool)
        S_filter[sub_edge_mask] = True
        S_filter[(sub_edge_mask) & (np.random.random(sub_edge_mask.shape[0]) > 0.5)] = False
        S_filter[(test_data.edge_index[0] == neighbor)] = False
        temp_data_1 = Data(x=test_data.x, edge_index=test_data.edge_index[:, S_filter], edge_label_index=temp_edge_label_index)
        S_filter[(test_data.edge_index[0] == neighbor)] = True
        temp_data_2 = Data(x=test_data.x, edge_index=test_data.edge_index[:, S_filter], edge_label_index=temp_edge_label_index)
        data_arr.extend([temp_data_1, temp_data_2])
        
preds = []
data_loader = DataLoader(data_arr, batch_size=128, shuffle=False)
for batch in data_loader:
    z = model.encode(batch.x, batch.edge_index)
    pred = model.decode(z, batch.edge_label_index)
    pred = pred.squeeze().cpu().detach().tolist()
    preds.extend(pred)
    
for i, neighbor in enumerate(neighbors):
    T = 100
    curr_preds = preds[2 * T * i:2 * T * (i + 1)]
    pred_diffs = [curr_preds[2 * j + 1] - curr_preds[2 * j] for j in range(T)]
    diff_avg, diff_std = sum(pred_diffs) / len(pred_diffs), statistics.stdev(pred_diffs) / np.sqrt(T)
    print(neighbor, "\t", round(diff_avg, 5), "\t", round(diff_std, 5), "\t", round(diff_avg / diff_std, 5))
    
print(datetime.now() - start)

C:\Users\Abhijit\AppData\Local\Temp\ipykernel_4984\886932573.py:15: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorAdvancedIndexing.cpp:1582.)
  S_filter[(sub_edge_mask) & (np.random.random(sub_edge_mask.shape[0]) > 0.5)] = False


256 	 47.34591 	 2.0414 	 23.19289
257 	 9.22889 	 2.59578 	 3.55535
2 	 38.49619 	 1.93213 	 19.9242
8 	 -73.15079 	 3.05025 	 -23.98188
12 	 -28.17014 	 1.70598 	 -16.51257
269 	 -6.24218 	 0.62798 	 -9.94005
270 	 245.84079 	 7.37823 	 33.31973
271 	 -12.94925 	 1.76703 	 -7.32827
273 	 -13.19283 	 1.49436 	 -8.82842
275 	 59.13121 	 2.17673 	 27.16512
276 	 -54.59223 	 2.97681 	 -18.33914
20 	 -16.43155 	 3.30116 	 -4.97751
279 	 -49.50857 	 2.38885 	 -20.72486
25 	 -73.75965 	 3.43174 	 -21.4934
282 	 -4.46916 	 0.53034 	 -8.42699
284 	 428.3876 	 10.79991 	 39.66584
29 	 -18.76953 	 1.51875 	 -12.35855
30 	 -4.46488 	 1.45683 	 -3.06478
289 	 -3.96742 	 2.53627 	 -1.56427
290 	 36.87545 	 2.89856 	 12.72197
38 	 -17.92081 	 1.0669 	 -16.79702
39 	 13.01413 	 5.39523 	 2.41215
296 	 17.88263 	 4.84899 	 3.68791
303 	 -50.8671 	 2.2852 	 -22.25933
50 	 -9.01994 	 0.70616 	 -12.7733
55 	 503.88052 	 10.39109 	 48.49159
314 	 -0.40873 	 3.20536 	 -0.12752
64 	 2.03439 	 0.80847 	 2.5